In [1]:
import re
import json
import codecs

from datetime import datetime
from collections import Counter, OrderedDict
from operator import itemgetter

import numpy as np
import pandas as pd

import gmplot
import geopy.distance

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
dtypes = {
    'amount': np.float32,
    'atm_address': str,
    'atm_address_lat': np.float32,
    'atm_address_lon': np.float32,
    'city': str,
    'country': str,
    'currency': np.float32,
    'customer_id': str,
    'home_add_lat': np.float32,
    'home_add_lon': np.float32,
    'mcc': str,
    'pos_address': str,
    'pos_address_lat': np.float32,
    'pos_address_lon': np.float32,
    'terminal_id': str,
    'transaction_date': str,
    'work_add_lat': np.float32,
    'work_add_lon': np.float32
}

In [3]:
use_columns = dtypes.keys()

df_1 = pd.read_csv("data/sets/train_set_mod_cities.csv",
                   sep=',', encoding='utf-8', dtype=dtypes, usecols=use_columns)
df_1["is_train"] = True
df_1.head()

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,work_add_lat,work_add_lon,is_train
0,2.884034,NaN,NaN,NaN,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5261,NaN,59.844074,30.179153,11606fde0c814ce78e0d726e39a0a5ee,2017-07-15,59.847,30.177,True
1,2.775633,NaN,NaN,NaN,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5261,NaN,59.844074,30.179153,e9647a5e1eacfb06713b6af755ccc595,2017-10-27,59.847,30.177,True
2,3.708368,NaN,NaN,NaN,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5992,"PR.MARSHALA ZHUKOVA,31St Petersburg190000 7...",59.858200,30.229023,df06c1fcd3718a514535ae822785f716,2017-10-03,59.847,30.177,True
3,2.787498,NaN,NaN,NaN,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5261,NaN,59.844074,30.179153,6c5e5793ebc984fb72875feffff62854,2017-09-09,59.847,30.177,True
4,2.892510,NaN,NaN,NaN,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5261,NaN,59.844074,30.179153,0576445d74e374c92c0902e612fca356,2017-07-06,59.847,30.177,True


In [4]:
use_columns = filter(lambda k: not (k.startswith('home_') or k.startswith('work_')), dtypes.iterkeys())

df_2 = pd.read_csv("data/sets/test_set_mod_cities.csv",
                   sep=',', encoding='utf-8', dtype=dtypes, usecols=use_columns)
df_2["mcc"] = df_2["mcc"].map(lambda x: re.sub(',', '', x))
df_2["is_train"] = False
df_2.head()

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,mcc,pos_address,pos_address_lat,pos_address_lon,terminal_id,transaction_date,is_train
0,2.211818,NaN,NaN,NaN,IVANTEEVKA,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,5411,2-1 TOLMACHEVA STRIVANTEEVKA141280 RUSRUS,55.967487,37.913681,ff0476dae4b098a7b16aabe93d4268df,2017-08-24,False
1,1.331379,NaN,NaN,NaN,IVANTEEVKA,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,4111,"SOVETSKIJ, 32IVANTEEVKA141282 RUSRUS",55.971294,37.905186,7cfd9a60282459d4692ecc85b856072e,2017-08-12,False
2,2.608004,NaN,NaN,NaN,PUSHKINO,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,5411,"105,KRASOARMEYSKOE SHPUSHKINO141206 RUSRUS",56.016590,37.909100,7e5a532f0029861d8a9c4f0479b9450b,2017-06-17,False
3,1.916752,NaN,NaN,NaN,IVANTEEVKA,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,5411,"G. IVANTEEVKA, UL.TOLMACHEVA, D.6IVANTEEVKA141...",55.964508,37.937912,2afe7d1bc61b86c449f413bdf2119032,2017-08-12,False
4,1.981067,NaN,NaN,NaN,MOSKVA,RUS,643.0,00fd410f5c580c8351cafa88d82b60f3,5814,5 KOMSOMOLSKAYA SQMOSCOW101000 RUSRUS,55.776802,37.657352,ab4f00601ff1d949afc59ee3f804c79c,2017-04-26,False


In [5]:
df = pd.concat([df_1, df_2], axis=0)
df["mcc"] = df["mcc"].astype(np.int32)

In [6]:
cols_drop = [
    'atm_address', 'atm_address_lat', 'atm_address_lon',
    'pos_address', 'pos_address_lat', 'pos_address_lon'
]
df.drop(labels=cols_drop, axis=1, inplace=True)

## Добавляем новые признаки, построенные на более ранних этапах

### 1. Признаки для atm-терминалов (банкоматы)

In [7]:
df_atm_features = pd.read_csv('data/atm_features.csv', sep=',', encoding='utf-8')
df_atm_features["is_atm"] = True
df_atm_features["is_raiff"] = df_atm_features["bank"] == u'Райффайзенбанк'
df_atm_features["is_partner"] = df_atm_features.apply(
    lambda x: x["bank"] not in {u'Райффайзенбанк', u'unknown'}, axis=1)
df_atm_features.drop(["bank", "city"], axis=1, inplace=True)
df_atm_features.rename(columns={"n_banks": "n_points"}, inplace=True)
df_atm_features.head()

,terminal_id,access24h,is_office,lat,lon,n_points,is_atm,is_raiff,is_partner
0,c693dcbafb5e0c1f9a58ac0211d79ed8,False,True,42.056451,48.298047,4,True,False,True
1,3ecc362617966cb7f4563c9e4e89b8a1,True,True,42.056451,48.298047,4,True,False,True
2,cd614b782e35146bacbad19e800e0624,False,True,42.056451,48.298047,4,True,False,True
3,fca99d9df0e01bdebca13824e1db1dfe,False,True,42.056451,48.298047,4,True,False,True
4,fabf56c3e89566e248ea94636cead5cb,False,False,42.061501,48.290501,5,True,False,False


In [8]:
df = pd.merge(df, df_atm_features, on='terminal_id', how='left', suffixes=('', '_atm'))
df[["access24h", "is_office", "is_atm", "is_raiff", "is_partner"]] = \
   df[["access24h", "is_office", "is_atm", "is_raiff", "is_partner"]].fillna(False)

### 2. Признаки для pos-терминалов

In [9]:
df_pos_features = pd.read_csv('data/pos_features.csv', sep=',', encoding='utf-8')
df_pos_features["pct_same"] = df_pos_features.apply(
    lambda x: x["n_points_same"] / float(x["n_points"]), axis=1)
df_pos_features.drop(['n_points_same', 'mcc'], axis=1, inplace=True)
df_pos_features.head()

,lat,lon,n_points,terminal_id,pct_same
0,42.102032,48.275082,9,35cbf8977b0279ea11c9f0c2bb4023f8,0.333333
1,42.102032,48.275082,9,4ed14e29988ba900f4606b7672ee78d3,0.111111
2,42.102032,48.275082,9,6ce9b191529e47eb9ac10e87e2c14033,0.444444
3,42.102032,48.275082,9,7048cc76efcc83cd740fa61ee623abd3,0.111111
4,42.102032,48.275082,9,a163fab651d6ce0c9bfa7b3e305ed731,0.444444


In [10]:
df = pd.merge(df, df_pos_features, on='terminal_id', how='left', suffixes=('', '_pos'))

df["pct_same"].fillna(1.0, inplace=True)

mask = df[["lat", "lon"]].isnull().any(axis=1)
df.loc[mask, ["lat", "lon"]] = df.loc[mask, ["lat_pos", "lon_pos"]].values

mask = df["n_points"].isnull()
df.loc[mask, "n_points"] = df.loc[mask, "n_points_pos"].values

df.drop(["lat_pos", "lon_pos", "n_points_pos"], axis=1, inplace=True)

### 3. Признаки для mcc-кодов

In [11]:
df_mcc_features = pd.read_csv('data/mcc_features.csv', sep=',', encoding='utf-8')
df_mcc_features.rename(columns={
        'n_home': 'neg_home',
        'n_work': 'neg_work',
        'p_home': 'pos_home',
        'p_work': 'pos_work'
    }, inplace=True)
df_mcc_features.head()

,mcc,neg_home,neg_work,pos_home,pos_work,mcc_group
0,50,False,False,False,False,0
1,146,False,False,False,False,0
2,160,False,False,False,False,0
3,165,False,False,False,False,0
4,168,False,False,False,False,0


In [12]:
df = pd.merge(df, df_mcc_features, on='mcc', how='left', suffixes=('', '_mcc'))

# Заполняем информацию для pos-терминалов
df[["neg_home", "neg_work", "pos_home", "pos_work"]] = \
    df[["neg_home", "neg_work", "pos_home", "pos_work"]].fillna(False)
df["mcc_group"].fillna(df["mcc_group"].max() + 1, inplace=True)
df["mcc_group"] = df["mcc_group"].astype(np.int32)

### 4. Признаки для клиентов

In [13]:
df_customers_features = pd.read_csv('data/customers_features.csv', sep=',', encoding='utf-8')
df_customers_features.head()

,customer_id,has_car,has_children,has_pet,pct_mcc_g00,pct_mcc_g01,pct_mcc_g02,pct_mcc_g03,pct_mcc_g04,pct_mcc_g05,...,pct_mcc_g11,pct_mcc_g12,pct_mcc_g13,pct_mcc_g14,pct_mcc_g15,pct_mcc_g16,pct_mcc_g17,pct_mcc_g18,pct_mcc_g19,pct_mcc_g20
0,0001f322716470bf9bfc1708f06f00fc,True,False,False,0.680,0.0,0.070,0.0,0.000,0.000,...,0.0,0.000,0.0,0.000,0.000,0.050,0.0,0.190,0.000,0.0
1,00021683ccb416637fe9a4cd35e4606e,False,True,True,0.064,0.0,0.000,0.0,0.000,0.013,...,0.0,0.013,0.0,0.026,0.000,0.359,0.0,0.436,0.000,0.0
2,0002d0f8a642272b41c292c12ab6e602,False,False,False,0.143,0.0,0.000,0.0,0.000,0.000,...,0.0,0.000,0.0,0.000,0.000,0.195,0.0,0.662,0.000,0.0
3,0004d182d9fede3ba2534b2d5e5ad27e,False,False,False,0.766,0.0,0.000,0.0,0.000,0.000,...,0.0,0.000,0.0,0.019,0.000,0.038,0.0,0.133,0.006,0.0
4,0007297d86e14bd68bd87b1dbdefe302,True,False,False,0.279,0.0,0.004,0.0,0.008,0.000,...,0.0,0.000,0.0,0.008,0.008,0.377,0.0,0.275,0.012,0.0


In [14]:
cols = ['customer_id', 'has_car', 'has_children', 'has_pet']
df = pd.merge(df, df_customers_features[cols], on='customer_id', how='left', suffixes=('', '_cust'))

In [15]:
df_customers_features = df.loc[:, ['customer_id', 'amount']].\
    groupby('customer_id', as_index=False).mean()
df_customers_features.rename(columns={'amount': 'amount_mean'}, inplace=True)
df_customers_features.head()

,customer_id,amount_mean
0,0001f322716470bf9bfc1708f06f00fc,3.198492
1,00021683ccb416637fe9a4cd35e4606e,2.965709
2,0002d0f8a642272b41c292c12ab6e602,2.361188
3,0004d182d9fede3ba2534b2d5e5ad27e,3.092196
4,0007297d86e14bd68bd87b1dbdefe302,2.808764


In [16]:
df = pd.merge(df, df_customers_features, on='customer_id', how='left', suffixes=('', '_cust'))
df["amount_sub"] = df["amount"] - df["amount_mean"]
df["amount_div"] = df["amount"] / df["amount_mean"]

### 5. Признаки для даты транзакций

In [17]:
df_date_features = pd.read_csv('data/date_features.csv', sep=',', encoding='utf-8')
df_date_features.head()

,day,day_of_week,days_after_holiday,days_before_holiday,is_dayoff,is_holiday,is_short,is_weekend,month,timestamp,transaction_date,week_of_year
0,27,5,20,18,False,False,False,False,1,1485464400,2017-01-27,4
1,28,6,21,17,False,False,False,True,1,1485550800,2017-01-28,4
2,29,7,22,16,False,False,False,True,1,1485637200,2017-01-29,4
3,30,1,23,15,False,False,False,False,1,1485723600,2017-01-30,5
4,31,2,24,14,False,False,False,False,1,1485810000,2017-01-31,5


In [18]:
df = pd.merge(df, df_date_features, on='transaction_date', how='left', suffixes=('', '_date'))

## Добавляеем парные признаки

### 6. Парные признаки для customer_id и terminal_id

In [19]:
df_cust_term = df[["customer_id", "terminal_id", "amount"]].\
    groupby(["customer_id", "terminal_id"], as_index=False).count()
df_cust_term.head()

,customer_id,terminal_id,amount
0,0001f322716470bf9bfc1708f06f00fc,0ea29ee7e6648a6a95ea186480bb14ef,15
1,0001f322716470bf9bfc1708f06f00fc,20ff27e387354f75e378ee7cf98637e8,1
2,0001f322716470bf9bfc1708f06f00fc,39ae766e056f616a16007b0fe906d663,49
3,0001f322716470bf9bfc1708f06f00fc,3abef409fd63a70cad2102e84854d61a,2
4,0001f322716470bf9bfc1708f06f00fc,4179f94a0fb689cbcb96b99972baac8a,1


In [20]:
df_cust = df_cust_term.groupby("customer_id", as_index=False).sum()
df_cust["amount"] = df_cust["amount"].astype(np.float32)
df_cust.head()

,customer_id,amount
0,0001f322716470bf9bfc1708f06f00fc,100.0
1,00021683ccb416637fe9a4cd35e4606e,78.0
2,0002d0f8a642272b41c292c12ab6e602,77.0
3,0004d182d9fede3ba2534b2d5e5ad27e,158.0
4,0007297d86e14bd68bd87b1dbdefe302,247.0


In [21]:
df_cust_term = pd.merge(df_cust_term, df_cust, how='left', on='customer_id', suffixes=('', '_total'))

df_cust_term["pct_term_counts"] = df_cust_term["amount"] / df_cust_term["amount_total"]
df_cust_term.drop(["amount", "amount_total"], axis=1, inplace=True)
df_cust_term.head()

,customer_id,terminal_id,pct_term_counts
0,0001f322716470bf9bfc1708f06f00fc,0ea29ee7e6648a6a95ea186480bb14ef,0.15
1,0001f322716470bf9bfc1708f06f00fc,20ff27e387354f75e378ee7cf98637e8,0.01
2,0001f322716470bf9bfc1708f06f00fc,39ae766e056f616a16007b0fe906d663,0.49
3,0001f322716470bf9bfc1708f06f00fc,3abef409fd63a70cad2102e84854d61a,0.02
4,0001f322716470bf9bfc1708f06f00fc,4179f94a0fb689cbcb96b99972baac8a,0.01


In [22]:
df = pd.merge(df, df_cust_term, on=['customer_id', 'terminal_id'], how='left', suffixes=('', '_cust_term'))

### 7. Парные признаки для customer_id и mcc_group

In [23]:
df_cust_mcc = df[["customer_id", "mcc_group", "amount"]].\
    groupby(["customer_id", "mcc_group"], as_index=False).count()
df_cust_mcc.head()

,customer_id,mcc_group,amount
0,0001f322716470bf9bfc1708f06f00fc,2,7
1,0001f322716470bf9bfc1708f06f00fc,9,1
2,0001f322716470bf9bfc1708f06f00fc,16,5
3,0001f322716470bf9bfc1708f06f00fc,18,19
4,0001f322716470bf9bfc1708f06f00fc,21,68


In [24]:
df_cust = df_cust_mcc.groupby("customer_id", as_index=False).sum()
df_cust["amount"] = df_cust["amount"].astype(np.float32)
df_cust.drop(labels='mcc_group', axis=1, inplace=True)
df_cust.head()

,customer_id,amount
0,0001f322716470bf9bfc1708f06f00fc,100.0
1,00021683ccb416637fe9a4cd35e4606e,78.0
2,0002d0f8a642272b41c292c12ab6e602,77.0
3,0004d182d9fede3ba2534b2d5e5ad27e,158.0
4,0007297d86e14bd68bd87b1dbdefe302,247.0


In [25]:
df_cust_mcc = pd.merge(df_cust_mcc, df_cust, how='left', on='customer_id', suffixes=('', '_total'))

df_cust_mcc["pct_mcc_g_counts"] = df_cust_mcc["amount"] / df_cust_mcc["amount_total"]
df_cust_mcc.drop(["amount", "amount_total"], axis=1, inplace=True)
df_cust_mcc.head()

,customer_id,mcc_group,pct_mcc_g_counts
0,0001f322716470bf9bfc1708f06f00fc,2,0.07
1,0001f322716470bf9bfc1708f06f00fc,9,0.01
2,0001f322716470bf9bfc1708f06f00fc,16,0.05
3,0001f322716470bf9bfc1708f06f00fc,18,0.19
4,0001f322716470bf9bfc1708f06f00fc,21,0.68


In [26]:
df = pd.merge(df, df_cust_mcc, on=["customer_id", "mcc_group"], how='left', suffixes=('', '_cust_mcc_g'))

### 8. Парные признаки для customer_id и date

In [27]:
df_mcc_features.set_index('mcc', drop=False, inplace=True)

In [28]:
holiday_groups = set(df_mcc_features.loc[[3501, 7512], "mcc_group"])

df_cust_date_features = df[["customer_id", "transaction_date", "country", "mcc", "mcc_group"]]
df_cust_date_features.drop_duplicates(inplace=True)

def is_away(country, mcc, mcc_g):
    answer = False
    
    if country != 'RUS':
        answer = True
    else:
        if mcc_g in holiday_groups:
            answer = True
        elif mcc in { 5309 }:
            answer = True
    return answer

df_cust_date_features["is_abroad"] = df_cust_date_features.apply(
    lambda x: is_away(x.loc["country"], x.loc["mcc"], x.loc["mcc_group"]), axis=1)
df_cust_date_features.drop(labels=["country", "mcc", "mcc_group"], axis=1, inplace=True)
df_cust_date_features.head()

/home/vladimir/anaconda2/lib/python2.7/site-packages/pandas/util/decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)
/home/vladimir/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/vladimir/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,customer_id,transaction_date,is_abroad
0,0dc0137d280a2a82d2dc89282450ff1b,2017-07-15,False
1,0dc0137d280a2a82d2dc89282450ff1b,2017-10-27,False
2,0dc0137d280a2a82d2dc89282450ff1b,2017-10-03,False
3,0dc0137d280a2a82d2dc89282450ff1b,2017-09-09,False
4,0dc0137d280a2a82d2dc89282450ff1b,2017-07-06,False


In [29]:
df = pd.merge(df, df_cust_date_features, on=['customer_id', 'transaction_date'],
              how='left', suffixes=('', '_cust_date'))

## Добавляем новые признаки

### 9. "Истинное" значение цены

In [30]:
df["amount_true"] = df["amount"].apply(lambda x: 2.5 * 10 ** x)

### 10. "Родной" город

In [31]:
mask_russian = df["country"] == "RUS"

map_cust_city = {
    cust_id: group.loc[:, "city"].value_counts().idxmax()
    for cust_id, group in df.loc[mask_russian, ["customer_id", "city"]].groupby("customer_id")
}

is_native = df.loc[:, "country"] == "RUS"
is_native = np.logical_and(is_native,
    df.loc[:, "mcc_group"] != df_mcc_features.loc[7011, "mcc_group"])
is_native = np.logical_and(is_native,
    df.loc[:, "customer_id"].apply(lambda x: map_cust_city[x]) == df.loc[:, "city"])
df["is_native"] = is_native

In [32]:
df.head()

,amount,city,country,currency,customer_id,home_add_lat,home_add_lon,is_train,mcc,terminal_id,...,is_short,is_weekend,month,timestamp,week_of_year,pct_term_counts,pct_mcc_g_counts,is_abroad,amount_true,is_native
0,2.884034,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5261,11606fde0c814ce78e0d726e39a0a5ee,...,False,True,7.0,1.500066e+09,28.0,0.013889,0.138889,False,1914.141006,True
1,2.775633,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5261,e9647a5e1eacfb06713b6af755ccc595,...,False,False,10.0,1.509052e+09,43.0,0.013889,0.138889,False,1491.327772,True
2,3.708368,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5992,df06c1fcd3718a514535ae822785f716,...,False,False,10.0,1.506978e+09,40.0,0.013889,0.013889,False,12773.452874,True
3,3.708368,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5992,df06c1fcd3718a514535ae822785f716,...,False,False,10.0,1.506978e+09,40.0,0.013889,0.013889,False,12773.452874,True
4,2.787498,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5261,6c5e5793ebc984fb72875feffff62854,...,False,True,9.0,1.504904e+09,36.0,0.013889,0.138889,False,1532.631571,True


## Постпроцессинг

Удалим строки с пустыми значениями долготы и широты

In [33]:
df = df.loc[df[["lat", "lon"]].notnull().all(axis=1)]

Заполним пропуски в поле city

In [34]:
df["city"].fillna('', inplace=True)

Уменьшим число уникальных городов, объединим самые маленькие в одну группу

In [35]:
mask_russian = df["country"] == "RUS"

r = [(c, g.shape[0]) for c, g in df[mask_russian].groupby("city")]
r = sorted(r, key=itemgetter(1), reverse=True)

In [36]:
for c, g in r:
    print "'{}'\t{}".format(c, g)

'MOSKVA'	1701745
'ST PETERBURG'	942300
'EKATERINBURG'	86483
'NOVOSIBIRSK'	84114
'CHEREPOVETS'	83872
'NIZHNIJ NOVGOROD'	82904
'KRASNOJARSK'	59223
'KRASNODAR'	54377
'SAMARA'	39203
'KHIMKI'	35823
'KAZAN'	33355
'SOCHI'	32578
'PODOLSK'	29846
'SYKTYVKAR'	28682
'MYTISHCHI'	26527
'UFA'	25655
'NOVOROSSIJSK'	25542
'PETROZAVODSK'	25377
'CHELJABINSK'	25182
'OREL'	24903
'VORONEZH'	23409
'JAROSLAVL'	21694
'OMSK'	21443
'ROSTOV NA DONU'	20464
'ODINTSOVO'	19970
'KALUGA'	19219
'BALASHIKHA'	18083
'PERM'	17058
'KRASNOGORSK'	16981
'ANAPA'	16398
'KIROV'	16048
'LJUBERTSY'	15547
'VOLGOGRAD'	15524
'KALININGRAD'	14215
'DOMODEDOVO'	13705
'TULA'	13648
'SARATOV'	13590
'TJUMEN'	13503
'IRKUTSK'	13494
'KOTELNIKI'	13368
'SURGUT'	12951
'KEMEROVO'	12270
'LIPETSK'	11482
'BRJANSK'	11377
'SARANSK'	10907
'KOROLEV'	10502
'VSEVOLOZHSK'	10274
'ZHELEZNODOROZHNYJ'	9149
'SMOLENSK'	8868
'PUSHKINO'	8809
'KURSK'	8670
'REUTOV'	8111
'LOBNJA'	7843
'BELGOROD'	7698
'TOMSK'	7138
'ZHELEZNOGORSK'	7113
'GATCHINA'	6496
'RAMENSKOE'	6433
'KOLPI

In [37]:
len(filter(lambda x: x[1] > 50, r)), df["city"].unique().shape[0]

(944, 3011)

In [38]:
small_cities_set = set(map(itemgetter(0), filter(lambda x: x[1] <= 50, r)))

In [39]:
df['city'] = df['city'].apply(
    lambda x: '__SMALL_TOWN__' if x in small_cities_set else x)
# df['city'] = df['city'].factorize()[0].astype(np.int32)

In [40]:
df.head()

,amount,city,country,currency,customer_id,home_add_lat,home_add_lon,is_train,mcc,terminal_id,...,is_short,is_weekend,month,timestamp,week_of_year,pct_term_counts,pct_mcc_g_counts,is_abroad,amount_true,is_native
0,2.884034,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5261,11606fde0c814ce78e0d726e39a0a5ee,...,False,True,7.0,1.500066e+09,28.0,0.013889,0.138889,False,1914.141006,True
1,2.775633,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5261,e9647a5e1eacfb06713b6af755ccc595,...,False,False,10.0,1.509052e+09,43.0,0.013889,0.138889,False,1491.327772,True
2,3.708368,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5992,df06c1fcd3718a514535ae822785f716,...,False,False,10.0,1.506978e+09,40.0,0.013889,0.013889,False,12773.452874,True
3,3.708368,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5992,df06c1fcd3718a514535ae822785f716,...,False,False,10.0,1.506978e+09,40.0,0.013889,0.013889,False,12773.452874,True
4,2.787498,ST PETERBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5261,6c5e5793ebc984fb72875feffff62854,...,False,True,9.0,1.504904e+09,36.0,0.013889,0.138889,False,1532.631571,True


Категоризуем переменные city и country

In [41]:
df['city'] = df['city'].factorize()[0].astype(np.int32)
df['country'] = df['country'].factorize()[0].astype(np.int32)

In [42]:
r = [(c, g.shape[0]) for c, g in df.groupby("city")]
r = sorted(r, key=itemgetter(1), reverse=True)
r = map(itemgetter(0), r)

r = {c:i for i, c in enumerate(r)}
df['city'] = df['city'].map(lambda x: r[x])

Грамотно приведем типы данных

In [43]:
df.isnull().any(axis=0)

amount                 False
city                   False
country                False
currency               False
customer_id            False
home_add_lat            True
home_add_lon            True
is_train               False
mcc                    False
terminal_id            False
transaction_date       False
work_add_lat            True
work_add_lon            True
access24h              False
is_office              False
lat                    False
lon                    False
n_points               False
is_atm                 False
is_raiff               False
is_partner             False
pct_same               False
neg_home               False
neg_work               False
pos_home               False
pos_work               False
mcc_group              False
has_car                False
has_children           False
has_pet                False
amount_mean            False
amount_sub             False
amount_div             False
day                    False
day_of_week   

In [44]:
zip(df.columns, df.dtypes)

[(u'amount', dtype('float32')),
 (u'city', dtype('int64')),
 (u'country', dtype('int32')),
 (u'currency', dtype('float32')),
 (u'customer_id', dtype('O')),
 (u'home_add_lat', dtype('float64')),
 (u'home_add_lon', dtype('float64')),
 ('is_train', dtype('bool')),
 (u'mcc', dtype('int32')),
 (u'terminal_id', dtype('O')),
 (u'transaction_date', dtype('O')),
 (u'work_add_lat', dtype('float64')),
 (u'work_add_lon', dtype('float64')),
 (u'access24h', dtype('bool')),
 (u'is_office', dtype('bool')),
 (u'lat', dtype('float64')),
 (u'lon', dtype('float64')),
 ('n_points', dtype('float64')),
 ('is_atm', dtype('bool')),
 ('is_raiff', dtype('bool')),
 ('is_partner', dtype('bool')),
 ('pct_same', dtype('float64')),
 ('neg_home', dtype('bool')),
 ('neg_work', dtype('bool')),
 ('pos_home', dtype('bool')),
 ('pos_work', dtype('bool')),
 (u'mcc_group', dtype('int32')),
 (u'has_car', dtype('bool')),
 (u'has_children', dtype('bool')),
 (u'has_pet', dtype('bool')),
 ('amount_mean', dtype('float32')),
 ('amo

In [45]:
# булевы признаки

cols = [
    'access24h', 'is_office', 'is_atm', 'is_raiff', 'is_partner',
    'neg_home', 'neg_work', 'pos_home', 'pos_work',
    'has_car', 'has_children', 'has_pet', 'is_abroad',
    'is_dayoff', 'is_holiday', 'is_short', 'is_weekend',
    'is_native'
]

df[cols] = df[cols].astype(np.int32)

In [46]:
# целочисленные признаки

cols = [
    'n_points', 'day', 'day_of_week', 'days_after_holiday', 'days_before_holiday',
    'month', 'timestamp', 'week_of_year', 'currency'
]

df[cols] = df[cols].astype(np.int32)

In [47]:
df.drop(labels=['transaction_date'], inplace=True, axis=1)

In [48]:
df.head()

,amount,city,country,currency,customer_id,home_add_lat,home_add_lon,is_train,mcc,terminal_id,...,is_short,is_weekend,month,timestamp,week_of_year,pct_term_counts,pct_mcc_g_counts,is_abroad,amount_true,is_native
0,2.884034,1,0,643,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5261,11606fde0c814ce78e0d726e39a0a5ee,...,0,1,7,1500066000,28,0.013889,0.138889,0,1914.141006,1
1,2.775633,1,0,643,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5261,e9647a5e1eacfb06713b6af755ccc595,...,0,0,10,1509051600,43,0.013889,0.138889,0,1491.327772,1
2,3.708368,1,0,643,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5992,df06c1fcd3718a514535ae822785f716,...,0,0,10,1506978000,40,0.013889,0.013889,0,12773.452874,1
3,3.708368,1,0,643,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5992,df06c1fcd3718a514535ae822785f716,...,0,0,10,1506978000,40,0.013889,0.013889,0,12773.452874,1
4,2.787498,1,0,643,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,True,5261,6c5e5793ebc984fb72875feffff62854,...,0,1,9,1504904400,36,0.013889,0.138889,0,1532.631571,1


Сохраняем итоговые датасеты

In [49]:
df_1 = df[df["is_train"] == True]
df_1.drop(labels="is_train", inplace=True, axis=1)
df_1.head()

/home/vladimir/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,amount,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,terminal_id,work_add_lat,...,is_short,is_weekend,month,timestamp,week_of_year,pct_term_counts,pct_mcc_g_counts,is_abroad,amount_true,is_native
0,2.884034,1,0,643,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5261,11606fde0c814ce78e0d726e39a0a5ee,59.847,...,0,1,7,1500066000,28,0.013889,0.138889,0,1914.141006,1
1,2.775633,1,0,643,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5261,e9647a5e1eacfb06713b6af755ccc595,59.847,...,0,0,10,1509051600,43,0.013889,0.138889,0,1491.327772,1
2,3.708368,1,0,643,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5992,df06c1fcd3718a514535ae822785f716,59.847,...,0,0,10,1506978000,40,0.013889,0.013889,0,12773.452874,1
3,3.708368,1,0,643,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5992,df06c1fcd3718a514535ae822785f716,59.847,...,0,0,10,1506978000,40,0.013889,0.013889,0,12773.452874,1
4,2.787498,1,0,643,0dc0137d280a2a82d2dc89282450ff1b,59.851002,30.232,5261,6c5e5793ebc984fb72875feffff62854,59.847,...,0,1,9,1504904400,36,0.013889,0.138889,0,1532.631571,1


In [50]:
df_2 = df[df["is_train"] == False]
df_2.drop(labels="is_train", inplace=True, axis=1)
df_2.head()

/home/vladimir/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,amount,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,terminal_id,work_add_lat,...,is_short,is_weekend,month,timestamp,week_of_year,pct_term_counts,pct_mcc_g_counts,is_abroad,amount_true,is_native
2299289,2.211818,70,0,643,00fd410f5c580c8351cafa88d82b60f3,NaN,NaN,5411,ff0476dae4b098a7b16aabe93d4268df,NaN,...,0,0,8,1503522000,34,0.117647,0.500000,0,407.153551,1
2299290,1.331379,70,0,643,00fd410f5c580c8351cafa88d82b60f3,NaN,NaN,4111,7cfd9a60282459d4692ecc85b856072e,NaN,...,0,1,8,1502485200,32,0.029412,0.088235,0,53.619044,1
2299291,1.331379,70,0,643,00fd410f5c580c8351cafa88d82b60f3,NaN,NaN,4111,7cfd9a60282459d4692ecc85b856072e,NaN,...,0,1,8,1502485200,32,0.029412,0.088235,0,53.619044,1
2299292,2.608004,50,0,643,00fd410f5c580c8351cafa88d82b60f3,NaN,NaN,5411,7e5a532f0029861d8a9c4f0479b9450b,NaN,...,0,1,6,1497646800,24,0.029412,0.500000,0,1013.779780,0
2299293,1.916752,70,0,643,00fd410f5c580c8351cafa88d82b60f3,NaN,NaN,5411,2afe7d1bc61b86c449f413bdf2119032,NaN,...,0,1,8,1502485200,32,0.029412,0.500000,0,206.391756,1


In [51]:
df_1.to_csv('data/sets/train_set_final.csv', sep=',', encoding='utf-8', index=False)
df_2.to_csv('data/sets/test_set_final.csv', sep=',', encoding='utf-8', index=False)